In [1]:
%cd /gxfs_work/cau/sunms534/trading_bot/

/gxfs_work/cau/sunms534/trading_bot


/gxfs_work/cau/sunms534/.conda/envs/my_pytorch_env/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import pandas as pd 
import numpy as np
from src.config import config, MODEL_CONFIG, PREP_CONFIG
from transformers import AutoTokenizer

In [3]:
# 1. Taking a look at cleaned dataset

In [4]:
stripped = pd.read_parquet(config.data.news.stripped)

In [5]:
stripped.columns

Index(['time', 'stocks', 'title', 'body', 'intra_day_time', 'channels',
       'company_name', 'short_name', 'author', 'parsed_body', 'cls_token',
       'staleness'],
      dtype='object')

In [6]:
stripped[["title", "stocks", "parsed_body"]].tail()

,title,stocks,parsed_body
index,,,
4578113,Cybin Announces Renewed At-The-Market Equity P...,CYBN,the company announces renewed atthemarket eq...
4578114,Environmental Icon Dr. Jane Goodall's Roots & ...,TM,environmental icon dr jane goodalls roots sho...
4578116,Chindata Group Unveils Pioneering Full-Stack S...,CD,chindata group unveils pioneering fullstack so...
4578117,Orbital Infrastructure Group Inc. Intends to E...,OIG,the company inc intends to enter into purcha...
4578118,"IBRX FINAL DEADLINE TUESDAY: ROSEN, TRUSTED IN...",IBRX,ibrx final deadline tuesday rosen trusted inve...


In [ ]:
# 2. Take a look at input_ids and if tokenizer works correctly

In [12]:
title_inputs_ids = pd.read_parquet(config.data.news.title_only.input_ids)

In [13]:
ld = pd.read_parquet(config.data.learning_dataset)

In [14]:
#cleaned = ld = pd.read_parquet(config.data.merged) # requires more memory

In [3]:
tokenizer = AutoTokenizer.from_pretrained(PREP_CONFIG.tokenizer)

In [44]:
mask = ld.parsed_body.apply(lambda x: "today announced" in x)

In [47]:
ld = ld[mask]

In [60]:
n = np.random.randint(0, ld.shape[0])
index = ld.index[n]
print(index)
entry = title_inputs_ids.loc[index, :].values

print(ld.loc[index, "stocks"])
print(f"{tokenizer.decode(entry)} \n")
print(ld.loc[index, "parsed_body"])

for x in entry:
    print(f"{x} --- {tokenizer.decode(x)}")


3944556
BLK
[CLS] [UNK] canada announces october cash distributions for the [UNK] etfs [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

BlackRock® Canada Announces October Cash Distributions for the iShares® ETFs. the company Asset Management Canada Limited , an indirect, wholly owned subsidiary of the company , today announced the October 2020 cash distributions for the iShares ETFs listed on the TSX or NEO which pay on a monthly basis. Unitholders of record of a fund will receive cash distributions payable in respect of that fund. As a fiduciary to investors and a leading provider of financial technology, our clients turn to us for the solutions they need when planning for their most important goals. As , the firm managed approximately US$7.81 trillion in assets on behalf of investors worldwide. With more than twenty years of experience, a global line up of 900+ exchange traded funds (ETFs) and US$2.32 tri

# Average number of token in news

In [4]:
title_inputs_ids = pd.read_parquet(config.data.news.input_ids)


In [5]:
n_padding = title_inputs_ids[title_inputs_ids == 0].sum(axis=1)

# Predictions from Model

In [4]:
ld = pd.read_parquet(config.data.learning_dataset, columns=["z_score", "z_score_class", "parsed_body"])
title_inputs_ids = pd.read_parquet(config.data.news.title_only.input_ids)
masks = pd.read_parquet(config.data.news.title_only.masks)

In [5]:
ld_mini = ld.iloc[-10000:,:]

In [6]:
index = ld.index[0]

In [7]:
import torch
import torch.nn.functional as F
from tqdm.notebook import tqdm


In [21]:
from transformers import AutoTokenizer, BertForSequenceClassification, BertTokenizer
from transformers import pipeline

finbert = BertForSequenceClassification.from_pretrained("/gxfs_work/cau/sunms534/trading_bot/data/models/ProsusAI/finbert", num_labels=3, local_files_only=True)

prob = []
preds = []
actuals = []
with torch.no_grad():
    for index in tqdm(ld_mini.index):
        input_id = torch.tensor([title_inputs_ids.loc[index, :].tolist()], dtype=torch.int32)
        mask = torch.tensor([masks.loc[index, :].tolist()])
        result = finbert(input_id, mask)
        probs = F.softmax(result.logits).numpy()
        prob.append(probs.max())
        preds.append((probs.argmax()  + 1) % 2)
        actuals.append(ld.loc[index, "z_score_class"])
        #print(result)  #LABEL_0: neutral; LABEL_1: positive; LABEL_2: negative

  0%|          | 0/10000 [00:00<?, ?it/s]

/scratch/SlurmTMP/sunms534.10715314/ipykernel_1460224/3251329861.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = F.softmax(result.logits).numpy()


In [44]:
F.softmax(result.logits).numpy().argmax()

/scratch/SlurmTMP/sunms534.10715314/ipykernel_1460224/3628110247.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  F.softmax(result.logits).numpy().argmax()


0

In [75]:
tokenizer.decode(title_inputs_ids.loc[ld_mini.index[532]])

'[CLS] robotexts increase while robocalls drop for third consecutive month, according to robokiller insights [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [23]:
len(preds)

10000

In [58]:
df = pd.DataFrame(zip(preds,actuals, prob))

In [59]:
df.loc[:, 2].describe()

count    10000.000000
mean         0.825836
std          0.124616
min          0.384929
25%          0.757534
50%          0.875642
75%          0.923297
max          0.969893
Name: 2, dtype: float64

In [60]:
df = df[df.iloc[:, 2] > 0.95]

In [61]:
len(df)

227

In [62]:
(df.iloc[:, 0] == df.iloc[:, 1]).sum()

115

In [63]:
df.iloc[:, 1].value_counts()

1
1    119
2     65
0     43
Name: count, dtype: int64

In [64]:
df.iloc[:, 0].value_counts()

0
1    210
0     17
Name: count, dtype: int64

In [70]:
df.iloc[:, 2].idxmax()

532

In [72]:
df.loc[532, :]

0    0.000000
1    1.000000
2    0.969893
Name: 532, dtype: float64

In [74]:
ld.loc[ld_mini.index[532], "parsed_body"]

'Robotexts Increase While Robocalls Drop For Third Consecutive Month, According to Robokiller Insights. Americans received 14.2 billion robotexts and 4.6 billion robocalls in May, a 9% increase and 10% decrease, respectively, according to Robokiller Insights. The two categories continue to follow diverging paths — this marks the third consecutive month over month increase in SMS spam and the second consecutive month over month decrease in unwanted calls. Government anti robocall efforts continue to pay dividends The decrease in robocalls can be attributed largely to the government\'s successful efforts to crack down on specific high volume robocalls. Yet, although the quantity of robocalls is dropping, the damage they cause is escalating. Americans have lost $29 billion to robocalls already in 2023, outpacing 2022\'s record losses. Scammers are getting better results despite fewer calls, signifying they continue to refine their tactics and double down on scams that work. According to t

## Analyze impact of correction for standard deviation 

In [ ]:
import pandas as pd
from src.config import config, MODEL_CONFIG
import numpy as np

In [ ]:
dat: pd.DataFrame = pd.read_parquet(path=config.data.merged)

In [5]:
dat.columns

Index(['est_entry_time', 'est_exit_time', 'entry_time', 'exit_time', 'r',
       'r_spy', 'unadj_entry_open', 'entry_is_too_far_apart',
       'exit_is_too_far_apart', 'parsed_body', 'staleness', 'stocks',
       'std_252', 'dollar_volume', 'r_intra_(t-1)', 'unadj_open', 'cond_vola',
       'r_mkt_adj', 'z_score', 'z_score_class', 'sample_weights'],
      dtype='object')

In [9]:
X = dat[['r', 'r_spy', 'std_252', 'stocks', 'entry_time']]

In [12]:
X.loc[:, 'mkt_adj'] = X['r'] - X['r_spy']

In [15]:
X = X.dropna()

In [17]:
X.describe()

,r,r_spy,std_252,mkt_adj
count,2.337559e+06,2.337559e+06,2.337559e+06,2.337559e+06
mean,2.261430e-04,1.758870e-04,2.962545e-02,5.025593e-05
std,4.079177e-02,8.814592e-03,8.741242e-01,3.954428e-02
min,-8.825647e-01,-1.190273e-01,1.822642e-04,-8.848767e-01
25%,-1.253482e-02,-3.663569e-03,1.592303e-02,-1.129293e-02
50%,0.000000e+00,5.393307e-04,2.241463e-02,-1.961995e-04
75%,1.235995e-02,4.528339e-03,3.398496e-02,1.077793e-02
max,6.535211e+00,1.044661e-01,4.723933e+02,6.522799e+00


In [18]:
X.loc[:, 'z_score'] = X['mkt_adj'] / X['std_252']

/scratch/SlurmTMP/sunms534.10881239/ipykernel_2837370/2975249423.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.loc[:, 'z_score'] = X['mkt_adj'] / X['std_252']


In [20]:
X.sort_values('z_score', inplace=True)

/scratch/SlurmTMP/sunms534.10881239/ipykernel_2837370/373010617.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.sort_values('z_score', inplace=True)


In [21]:
X

,r,r_spy,std_252,stocks,entry_time,mkt_adj,z_score
index,,,,,,,
4254414,0.000084,0.020043,0.000191,FTSM,2022-02-25 09:31:00-05:00,-0.019959,-104.360953
4236591,0.000000,0.018497,0.000190,FTSM,2022-01-31 09:31:00-05:00,-0.018497,-97.491017
4428595,0.000168,0.028950,0.000473,FTSM,2022-11-30 09:31:00-05:00,-0.028781,-60.812962
1067216,-0.000168,0.010046,0.000210,FTSM,2022-03-24 09:31:00-04:00,-0.010214,-48.708962
4352005,-0.000673,0.011286,0.000301,FTSM,2022-07-29 09:31:00-04:00,-0.011959,-39.721556
...,...,...,...,...,...,...,...
3796745,2.669869,0.001551,0.040814,RCON,2019-12-26 11:06:00-05:00,2.668318,65.377072
1067218,0.000167,-0.012177,0.000182,FTSM,2021-12-03 09:31:00-05:00,0.012345,67.728700
4161429,0.000083,-0.014148,0.000202,FTSM,2021-09-30 09:31:00-04:00,0.014232,70.398063


In [28]:
X['z_score'].describe()

count    2.337559e+06
mean     1.480262e-04
std      1.107952e+00
min     -1.043610e+02
25%     -4.966595e-01
50%     -9.333168e-03
75%      4.859626e-01
max      1.195337e+02
Name: z_score, dtype: float64

In [29]:
X.loc[:, "z_score"] = X["z_score"].clip(lower=X["z_score"].quantile(0.05), upper=X["z_score"].quantile(0.95))

In [30]:
X['z_score'].describe()

count    2.337559e+06
mean    -2.617431e-03
std      7.952768e-01
min     -1.577498e+00
25%     -4.966595e-01
50%     -9.333168e-03
75%      4.859626e-01
max      1.602021e+00
Name: z_score, dtype: float64

Maximum ist 1.6, minimum ist -1.6 nach winsorisieren...

Problem, wenn wir mit std_252 standardisieren ist, dass es Wertpapiere (Aktien?) gibt mit sehr geringer Vola. Und weil wir die std von r nehmen, und nicht von r - r_spy bekommen wir unsinnige, viel zu große Werte heraus. -> Filtere minimale std_252 heraus.

In [31]:
X['std_252'].describe()

count    2.337559e+06
mean     2.962545e-02
std      8.741242e-01
min      1.822642e-04
25%      1.592303e-02
50%      2.241463e-02
75%      3.398496e-02
max      4.723933e+02
Name: std_252, dtype: float64

In [27]:
X.sort_values('std_252')

,r,r_spy,std_252,stocks,entry_time,mkt_adj,z_score
index,,,,,,,
1067218,0.000167,-0.012177,0.000182,FTSM,2021-12-03 09:31:00-05:00,0.012345,67.728700
4203383,-0.000084,-0.010544,0.000182,FTSM,2021-11-30 09:31:00-05:00,0.010461,57.334128
4180641,0.000084,0.006958,0.000185,FTSM,2021-10-29 09:31:00-04:00,-0.006875,-37.152865
4236591,0.000000,0.018497,0.000190,FTSM,2022-01-31 09:31:00-05:00,-0.018497,-97.491017
4254414,0.000084,0.020043,0.000191,FTSM,2022-02-25 09:31:00-05:00,-0.019959,-104.360953
...,...,...,...,...,...,...,...
3042698,0.018086,0.004291,472.393155,IDN,2015-04-22 09:42:00-04:00,0.013795,0.000029
3044272,0.012579,-0.003499,472.393159,IDN,2015-04-24 10:40:00-04:00,0.016077,0.000034
3029747,0.065789,0.001155,472.393187,IDN,2015-03-17 09:35:00-04:00,0.064634,0.000137


In [40]:
learndat: pd.DataFrame = pd.read_parquet(path=config.data.learning_dataset)

In [39]:
learndat.sort_values("z_score")

,est_entry_time,est_exit_time,entry_time,exit_time,r,r_spy,unadj_entry_open,entry_is_too_far_apart,exit_is_too_far_apart,parsed_body,...,std_252,dollar_volume,r_intra_(t-1),unadj_open,cond_vola,r_mkt_adj,z_score,z_score_class,sample_weights,split
index,,,,,,,,,,,,,,,,,,,,,
3537527,2018-06-15 09:31:00-04:00,2018-06-15 16:01:00-04:00,2018-06-15 09:31:00-04:00,2018-06-15 16:01:00-04:00,-0.054398,0.001953,49.45,False,False,Steel Dynamics Provides Second Quarter 2018 Ea...,...,0.018048,828416.205000,-0.003457,49.45,0.020124,-0.056352,-0.053765,0,0.765191,training
213410,2022-03-15 09:31:00-04:00,2022-03-15 16:01:00-04:00,2022-03-15 09:31:00-04:00,2022-03-15 16:01:00-04:00,-0.122239,0.013514,33.05,False,False,ANI Pharma Q4 Adj. Profit Declines. RTTNews) ...,...,0.043979,80336.931489,-0.021674,33.05,0.114946,-0.135753,-0.053765,0,0.277347,training
213386,2022-11-09 09:31:00-05:00,2022-11-09 16:01:00-05:00,2022-11-09 09:31:00-05:00,2022-11-09 16:01:00-05:00,-0.110143,-0.014353,39.04,False,False,ANI Pharmaceuticals (ANIP) Surpasses Q3 Earnin...,...,0.030984,33667.720674,0.005255,39.04,0.055847,-0.095790,-0.053765,0,0.263123,training
213385,2022-11-09 09:31:00-05:00,2022-11-09 16:01:00-05:00,2022-11-09 09:31:00-05:00,2022-11-09 16:01:00-05:00,-0.110143,-0.014353,39.04,False,False,"ANI Pharmaceuticals, Inc. Q3 Earnings Summary....",...,0.030984,33667.720674,0.005255,39.04,0.055847,-0.095790,-0.053765,0,0.263123,training
213384,2022-11-09 09:31:00-05:00,2022-11-09 16:01:00-05:00,2022-11-09 09:31:00-05:00,2022-11-09 16:01:00-05:00,-0.110143,-0.014353,39.04,False,False,ANI Pharmaceuticals Q3 22 Earnings Conference ...,...,0.030984,33667.720674,0.005255,39.04,0.055847,-0.095790,-0.053765,0,0.263123,training
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3477071,2018-02-20 09:31:00-05:00,2018-02-20 16:01:00-05:00,2018-02-20 09:31:00-05:00,2018-02-20 16:01:00-05:00,0.053998,-0.003123,48.52,False,False,"Standard Motor Products, Inc. to Host Earnings...",...,0.013925,35306.641478,0.002496,48.52,0.030848,0.057121,0.051480,1,0.665193,training
3477035,2018-02-20 09:31:00-05:00,2018-02-20 16:01:00-05:00,2018-02-20 09:31:00-05:00,2018-02-20 16:00:00-05:00,0.120614,-0.002719,18.24,False,False,Rayonier Advanced Materials Inc to Host Earnin...,...,0.026722,109168.955270,-0.000549,18.24,0.039932,0.123333,0.051480,1,0.697150,training
4577978,2023-08-24 09:31:00-04:00,2023-08-24 16:01:00-04:00,2023-08-24 09:31:00-04:00,2023-08-24 16:01:00-04:00,0.075122,-0.017811,30.75,False,False,Aviat Networks Announces Fiscal 2023 Fourth Qu...,...,0.026545,30494.502209,0.021459,30.75,0.052597,0.092933,0.051480,1,0.215614,testing


In [37]:
learndat.z_score.describe()

count    954478.000000
mean         -0.000081
std           0.021671
min          -0.053765
25%          -0.010286
50%          -0.000039
75%           0.010272
max           0.051480
Name: z_score, dtype: float64

In [38]:
(learndat.z_score/learndat.std_252).describe()

count    954478.000000
mean         -0.003496
std           0.949972
min        -104.360953
25%          -0.497133
50%          -0.002012
75%           0.500479
max         119.533666
dtype: float64